In [1]:
import sys
sys.path.insert(0, '../../../Utilities/')
import argparse
import os
import torch
from collections import OrderedDict
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from matplotlib.pyplot import savefig
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
import seaborn as sns
import pylab as py
import time
from doe_lhs import *
import warnings
sys.path.insert(0, '../../../Scripts/')
from models_pde import Net
from pinnn import *
# from ../Scripts/helper import *

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [3]:
num_epochs = 40000
lambda_phy = 1.5

noise = 0.0

## Network Architecture
hid_dim = 128
num_layer = 4


In [4]:
# Doman bounds
lb = np.array([-1.0, 0.0])
ub = np.array([1.0, 1.0])

N0 = 512
N_b = 100
N_f = 5000
# layers = [2, 100, 100, 100, 100, 2]

data = scipy.io.loadmat('AC.mat')

t = data['tt'].flatten()[:,None] # 201
x = data['x'].flatten()[:,None]  # 512
Exact = np.real(data['uu'])

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.T.flatten()[:,None]

###########################

st = 0.30
tt = int(t.shape[0] * st)


trunk1_X = X_star[ : x.shape[0] * tt]
trunk2_X = X_star[ x.shape[0] * (tt - 1) : ]

trunk1_u = u_star[ : x.shape[0] * tt]
trunk2_u = u_star[ x.shape[0] * (tt - 1) : ]

############################


first_lb = trunk1_X.min(0) # [-5.  0.]
first_ub = trunk1_X.max(0) # [4.9609375  0.77754418]

second_lb = trunk2_X.min(0) # [-5.       0.77754418]
second_ub = trunk2_X.max(0) # [4.9609375  1.57079633]

# --------------------- first half ---------------------

# initial points
idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0_1 = x[idx_x,:]
u0_1 = Exact[idx_x,0:1]

# boundary points
idx_t = np.random.choice(tt, N_b, replace=True)
tb_1 = t[idx_t,:]

# collocation points
X_f_1 = first_lb + (first_ub-first_lb)*lhs(2, N_f)


X0_1 = np.concatenate((x0_1, 0*x0_1), 1) # (x0, 0)
Y0_1 = u0_1 
X_lb_1 = np.concatenate((0*tb_1 + first_lb[0], tb_1), 1) # (lb[0], tb)
X_ub_1 = np.concatenate((0*tb_1 + first_ub[0], tb_1), 1) # (ub[0], tb)


In [5]:
net = Net(in_dim = 2, out_dim = 1, hid_dim = hid_dim, num_layers = num_layer).to(device)

ACs = AC_PINN(X0_1, Y0_1, X_f_1, X_lb_1, X_ub_1, net, device, num_epochs, lambda_phy, noise)
 
ACs.train()

[Epoch 0/40000] [MSE loss: 0.111202] [Phy loss: 0.520093] [Total loss: 11.902548]


KeyboardInterrupt: 

In [ ]:
nsamples = 1
u_pred_list = []
f_u_pred_list = []

for run in range(nsamples):
    y_pred, f_u_pred = ACs.get_residual(trunk1_X)
    u_pred = y_pred[:,0:1].detach().cpu().numpy()
    u_pred_list.append(u_pred)
    f_u_pred_list.append(f_u_pred.detach().cpu().numpy())


u_pred_arr = np.array(u_pred_list)
f_u_pred_arr = np.array(f_u_pred_list)

u_pred = u_pred_arr.mean(axis=0)
f_u_pred = f_u_pred_arr.mean(axis=0)



residual = (f_u_pred**2).mean()

#     u_dev = u_pred_arr.var(axis=0)
#     f_dev = f_pred_arr.var(axis=0)

error_u = np.linalg.norm(trunk1_u-u_pred,2)/np.linalg.norm(trunk1_u,2)

print("Error u:", error_u)                   
print('Residual: %e' % (residual))

# --------------------- Second half ---------------------

tmp_pred = u_pred[-512:]

In [ ]:
tmp_pred = u_pred[-512:]

# initial points
x0_2 = x[idx_x,:]
u0_2 = u_pred[-512:][idx_x,0:1]


# boundary points
idx_t = np.random.choice(range(tt, 201), N_b, replace=False)
tb_2 = t[idx_t,:]

# collocation points
X_f_2 = second_lb + (second_ub-second_lb)*lhs(2, N_f)

X0_2 = np.concatenate((x[idx_x,:], 
                       np.full(x[idx_x,:].shape, first_ub[1])), 1) # (x0, 0)



Y0_2 = u0_2
X_lb_2 = np.concatenate((0*tb_2 + second_lb[0], tb_2), 1) # (lb[0], tb)
X_ub_2 = np.concatenate((0*tb_2 + second_ub[0], tb_2), 1) # (ub[0], tb)


num_epochs = 20000

PINN = AC_PINN(np.vstack([X0_1,X0_2]),
                        np.vstack([Y0_1,Y0_2]),
                        np.vstack([X_f_1,X_f_2]), 
                        np.vstack([X_lb_1,X_lb_2]), 
                        np.vstack([X_ub_1,X_ub_2]), 
                        net, 
                        device, 
                        num_epochs, 
                        lambda_phy, 
                        noise)
PINN.train()

In [ ]:
nsamples = 1
u_pred_list = []
f_u_pred_list = []

for run in range(1):
    y_pred, f_u_pred = PINN.get_residual(X_star)
    u_pred = y_pred[:,0:1].detach().cpu().numpy()
    u_pred_list.append(u_pred)
    f_u_pred_list.append(f_u_pred.detach().cpu().numpy())


u_pred_arr = np.array(u_pred_list)
f_u_pred_arr = np.array(f_u_pred_list)


u_pred = u_pred_arr.mean(axis=0)
f_u_pred = f_u_pred_arr.mean(axis=0)
residual = (f_u_pred**2).mean()

error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)


U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
FU_pred = griddata(X_star, f_u_pred.flatten(), (X, T), method='cubic')

In [ ]:
print("Error u:", error_u)                  
print('Residual: %e' % (residual))

In [ ]:
## ############################# Plotting ###############################
######################################################################    
# t = data['tt'].flatten()[:,None]
# x = data['x'].flatten()[:,None]

# X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
# X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
# X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
# X_u_train = np.vstack([X0, X_lb, X_ub])

# fig, ax = newfig(1.0, 0.9)
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.axis('off')

####### Row 0: h(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/6, left=0.15, right=0.25, wspace=0)
ax = plt.subplot(gs0[:, :])

# h = ax.imshow(Exact, interpolation='nearest', cmap='YlGnBu', 
#               extent=[lb[1], ub[1], lb[0], ub[0]], 
#               origin='lower', aspect='auto')

h = ax.imshow(U_pred.T, interpolation='nearest', cmap='YlGnBu', 
              extent=[lb[1], ub[1], lb[0], ub[0]], 
              origin='lower', aspect='auto')


divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)

ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
#    plt.setp(leg.get_texts(), color='w')
ax.set_title('Prediction $u(x,t)$', fontsize = 10)
savefig('Prediction.svg',dpi=600,bbox_inches='tight')

fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111)
ax.axis('off')


####### Row 0: h(t,x) ##################    
gs0 = gridspec.GridSpec(1, 2)
gs0.update(top=1-0.06, bottom=1-1/6, left=0.15, right=0.25, wspace=0)
ax = plt.subplot(gs0[:, :])

# error  = griddata(X_star, np.abs(u_star-U_pred).flatten(), (X, T), method='cubic')


# h = ax.imshow(U_pred.T, interpolation='nearest', cmap='YlGnBu', 
#               extent=[lb[1], ub[1], lb[0], ub[0]], 
#               origin='lower', aspect='auto')




eq = (U_pred.T - Exact ) / (Exact)


h = ax.imshow(np.abs(Exact - U_pred.T) * 5, interpolation='nearest', cmap='YlGnBu', 
              extent=[lb[1], ub[1], lb[0], ub[0]], 
              origin='lower', aspect='auto')

# h = ax.imshow(np.abs(eq), interpolation='nearest', cmap='YlGnBu', 
#               extent=[lb[1], ub[1], lb[0], ub[0]], 
#               origin='lower', aspect='auto',vmax = 10,vmin = 0)

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(h, cax=cax)



ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
leg = ax.legend(frameon=False, loc = 'best')
#    plt.setp(leg.get_texts(), color='w')
ax.set_title('Error $u(x,t)$', fontsize = 10)
# savefig('Error.svg',dpi=600,bbox_inches='tight')

In [ ]:
## Row 1: h(t,x) slices ##################    
fig = plt.figure(figsize=(40, 20))
ax = fig.add_subplot(111)

# b = np.loadtxt('PINN_result.txt',dtype=float)

gs1 = gridspec.GridSpec(1, 4)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.3, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x,Exact[:,0], 'b-', linewidth = 2, label = 'Exact')       
# ax.plot(x,0, 'r--', linewidth = 2, label = 'Prediction')
ax.plot(x,U_pred.T[:,0], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
# ax.set_ylabel('$u(x,t)$')   
ax.set_title('t = 0.0', fontsize = 10)
ax.axis('square')
ax.set_xlim([-1.0,1.0])
ax.set_ylim([-1.0,1.0])   

ax = plt.subplot(gs1[0, 1])
ax.plot(x,Exact[:,50], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred.T[:,50], 'r--', linewidth = 2, label = 'Prediction')

ax.set_xlabel('$x$')
# ax.set_ylabel('$u(x,t)$')
ax.axis('square')
ax.set_xlim([-1.0,1.0])
ax.set_ylim([-1.0,1.0])  
ax.set_title('t = 0.25', fontsize = 10)


ax = plt.subplot(gs1[0, 2])
ax.plot(x,Exact[:,100], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred.T[:,100], 'r--', linewidth = 2, label = 'Prediction')

ax.set_xlabel('$x$')
# ax.set_ylabel('$u(x,t)$')
ax.axis('square')
ax.set_xlim([-1.0,1.0])
ax.set_ylim([-1.0,1.0])      
ax.set_title('t = 0.50', fontsize = 10)

ax.legend(loc='upper center', bbox_to_anchor=(-0.25, -0.2), ncol=1, frameon=True)

ax = plt.subplot(gs1[0, 3])
ax.plot(x,Exact[:,150], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,U_pred.T[:,150], 'r--', linewidth = 2, label = 'Prediction')

ax.set_xlabel('$x$')
# ax.set_ylabel('$u(x,t)$')
ax.axis('square')
ax.set_xlim([-1.0,1.0])
ax.set_ylim([-1.0,1.0])   
ax.set_title('t = 0.75', fontsize = 10)

savefig('times.svg',dpi=600,bbox_inches='tight')